# PoC

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

In [3]:
logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    stream=sys.stdout,
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

In [4]:
log = logging.getLogger(__name__)

In [5]:
from omegaconf import OmegaConf

In [6]:
c = OmegaConf.load("../config/main.yaml")

In [7]:
c.settings.debug = False
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"
c.settings.dirs.input_minimal = "../../datasets/inputs/"
c.settings.dirs.preprocess = "../../inputs/preprocess/"

In [8]:
log.info(OmegaConf.to_yaml(c))

2022-02-25 21:40:23,643 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
  group: default
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
    input_minimal: ../../datasets/inputs/
    feature: ${settings.dirs.input}features/
    preprocess: ../../inputs/preprocess/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
  training_method: nn
params:
  seed: 440
  n_class: 10
  preprocess:
  - remove_china_shock
  pca_n_components: 50
  n_fold: 5
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: combinational_purged
  group_name: in

In [9]:
import torch

In [10]:
import src.utils as utils

In [11]:
device = utils.gpu_settings(c)

2022-02-25 21:40:24,880 [INFO] [utils] CUDA_VISIBLE_DEVICES: 6,7
2022-02-25 21:40:24,917 [INFO] [utils] torch device: cuda, device count: 2


## Preprocess

In [12]:
from src.load_data import InputData

2022-02-25 21:40:25,240 [INFO] [loader] Loading faiss with AVX2 support.
2022-02-25 21:40:25,332 [INFO] [loader] Successfully loaded faiss with AVX2 support.


In [13]:
input = InputData(c)

2022-02-25 21:40:25,358 [INFO] [load_data] Load feather file. path: ../../inputs/train.f
2022-02-25 21:40:56,973 [INFO] [utils] Mem. usage decreased to 307Mb: 49% reduction
2022-02-25 21:40:57,005 [INFO] [load_data] Load feather file. path: ../../inputs/example_test.f
2022-02-25 21:40:57,084 [INFO] [utils] Mem. usage decreased to 0.0Mb: 49% reduction
2022-02-25 21:40:57,085 [INFO] [load_data] Load feather file. path: ../../inputs/example_sample_submission.f
2022-02-25 21:40:57,090 [INFO] [utils] Mem. usage decreased to 0.0Mb: 34% reduction


In [14]:
input.train.describe()

,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,f_6,...,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,group_fold,time_fold
count,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,...,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06,2.635488e+06
mean,7.103500e+02,1.885160e+03,-1.702157e-02,6.950274e-04,-8.909235e-03,6.209589e-03,-1.757490e-02,-3.142135e-03,-1.961225e-03,-3.933720e-04,...,-6.558229e-03,9.289950e-04,-2.182449e-03,-2.326412e-02,-1.609894e-03,-5.184209e-03,-4.299405e-03,-3.754060e-03,2.000000e+00,2.500000e+00
std,3.708622e+02,1.083865e+03,9.220569e-01,1.097723e+00,1.043660e+00,1.034713e+00,9.686490e-01,9.959193e-01,1.119593e+00,1.080068e+00,...,1.083052e+00,1.123110e+00,1.164162e+00,9.673325e-01,1.158741e+00,1.126991e+00,1.063325e+00,1.013074e+00,1.414214e+00,1.686267e+00
min,0.000000e+00,0.000000e+00,-9.419646e+00,-1.765789e+01,-6.579473e+00,-8.644268e+00,-1.240608e+01,-4.000015e+00,-8.833704e+00,-6.384251e+00,...,-8.183732e+00,-2.399478e+01,-8.911345e+00,-9.348986e+00,-7.366648e+00,-7.579406e+00,-6.707284e+00,-1.028264e+01,0.000000e+00,0.000000e+00
25%,2.930000e+02,9.510000e+02,-4.822457e-01,-4.268636e-01,-6.918773e-01,-6.546747e-01,-4.543629e-01,-3.231966e-01,-7.110226e-01,-8.051575e-01,...,-7.049787e-01,-2.051390e-01,-9.585723e-01,-5.027200e-01,-9.759416e-01,-7.407039e-01,-8.141529e-01,-5.032123e-01,1.000000e+00,1.000000e+00
50%,7.970000e+02,1.881000e+03,-8.900590e-02,2.451799e-01,-4.580044e-02,7.742916e-02,-2.804235e-01,-1.807949e-01,-1.967342e-02,6.589857e-03,...,-2.043986e-01,2.393769e-01,2.033623e-01,-2.872780e-01,7.143374e-03,-1.648386e-01,3.606625e-02,-2.855791e-01,2.000000e+00,2.500000e+00
75%,1.016000e+03,2.830000e+03,3.534553e-01,6.788467e-01,6.313426e-01,6.642895e-01,7.873586e-02,2.538166e-02,6.906360e-01,8.086394e-01,...,4.779136e-01,5.475935e-01,8.738781e-01,1.196992e-01,9.685071e-01,6.232187e-01,7.972904e-01,1.359206e-01,3.000000e+00,4.000000e+00
max,1.219000e+03,3.773000e+03,1.203861e+01,7.845261e+00,8.707207e+00,8.009340e+00,3.575379e+01,7.662866e+01,7.646200e+00,6.778142e+00,...,2.231185e+01,6.587691e+00,6.978151e+00,6.140367e+01,7.679950e+00,1.241804e+01,7.003982e+00,4.337021e+01,4.000000e+00,5.000000e+00


## Fold

In [15]:
input.train.fillna({"time_fold": c.params.n_fold}, inplace=True)

In [16]:
input.train

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,group_fold,time_fold
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566,4.0,0.0
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624,2.0,0.0
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608,0.0,0.0
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309,2.0,0.0
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635483,1219_3768,1219,3768,0.033600,0.093530,-0.720275,-0.345497,-0.438781,-0.166972,-0.437182,...,-0.660579,0.875537,0.421628,-0.428097,-0.075548,-0.533092,-0.193732,-0.581394,3.0,5.0
2635484,1219_3769,1219,3769,-0.223264,-1.344935,-0.199987,-0.107702,-0.454677,-0.221914,-0.141174,...,-0.670493,0.875537,0.421628,-0.729949,-1.514277,0.013145,-0.890270,-0.589705,3.0,5.0
2635485,1219_3770,1219,3770,-0.559415,0.979489,-1.110491,1.006980,-0.467307,-0.159549,1.355671,...,0.820784,-1.142157,0.421628,-0.363329,1.363181,-0.079106,-1.580124,-0.297625,1.0,5.0
2635486,1219_3772,1219,3772,0.009599,-2.565332,0.320301,0.076600,1.380182,-0.155366,-0.689000,...,0.133074,-1.142157,0.421628,-0.375288,-1.514277,-0.973762,0.608647,-0.372040,0.0,5.0


In [17]:
input.train["group_fold"].value_counts()

1.0    527098
4.0    527098
2.0    527098
3.0    527097
0.0    527097
Name: group_fold, dtype: int64

In [18]:
input.train["time_fold"].value_counts()

1.0    439248
5.0    439248
4.0    439248
3.0    439248
2.0    439248
0.0    439248
Name: time_fold, dtype: int64

In [19]:
input.train[["group_fold", "time_fold"]].value_counts()

group_fold  time_fold
1.0         0.0          89294
2.0         2.0          88471
3.0         1.0          88417
1.0         1.0          88367
4.0         0.0          88255
            3.0          88254
            5.0          88228
2.0         4.0          88211
3.0         4.0          88186
2.0         3.0          88158
3.0         3.0          88143
0.0         5.0          88142
            2.0          88079
3.0         5.0          88041
0.0         3.0          87807
            0.0          87758
1.0         2.0          87737
0.0         4.0          87725
            1.0          87586
4.0         4.0          87583
3.0         2.0          87583
2.0         5.0          87566
1.0         4.0          87543
2.0         1.0          87478
4.0         1.0          87400
            2.0          87378
1.0         5.0          87271
2.0         0.0          87214
1.0         3.0          86886
3.0         0.0          86727
dtype: int64

In [20]:
from src.make_fold import train_test_split

In [21]:
train_folds, valid_folds = train_test_split(c, input.train, 1)

2022-02-25 21:41:19,553 [INFO] [make_fold] Num of training data: 1753256, num of validation data: 878496


In [22]:
len(train_folds)

1753256

In [23]:
len(valid_folds)

878496

## Model

In [24]:
from src.make_model import make_model

In [25]:
# c.params.model = "ump_1"

In [26]:
model = make_model(c, device)

In [27]:
model

DataParallel(
  (module): LSTMModel(
    (bn_1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rnn): LSTM(900, 300, batch_first=True, bidirectional=True)
    (head): Linear(in_features=600, out_features=1, bias=True)
  )
)

In [28]:
input_ = torch.randn(c.params.batch_size, c.params.model_input * 10).to(device)
# input_ = torch.randn(c.params.batch_size, c.params.model_window, c.params.model_input).to(device)

# num_gpu = len(c.settings.gpus.split(","))
# hidden = torch.zeros(1, c.params.batch_size // num_gpu, 300).to(device)
# cell = torch.zeros(1, c.params.batch_size // num_gpu, 300).to(device)
# hidden_cell = (hidden, cell)

In [29]:
input_.size()

torch.Size([640, 9000])

In [30]:
res = model(input_)
# res, hidden_cell = model(input_, hidden_cell)
# res, hidden_cell = model(input_)

In [31]:
res.shape

torch.Size([640, 10])

In [32]:
res

tensor([[-0.0403,  0.0547,  0.2515,  ..., -0.1478, -0.1288, -0.2212],
        [-0.0569, -0.0569,  0.1770,  ...,  0.0724, -0.0057, -0.1610],
        [ 0.2192,  0.1237,  0.3071,  ..., -0.0595, -0.1566, -0.0493],
        ...,
        [-0.0457,  0.0742,  0.0134,  ...,  0.1462,  0.1443,  0.0099],
        [ 0.1139,  0.0947,  0.3130,  ..., -0.0405, -0.0991,  0.0658],
        [ 0.0894,  0.1619, -0.2130,  ...,  0.0745,  0.1992, -0.0068]],
       device='cuda:0', dtype=torch.float16, grad_fn=<GatherBackward>)